In [ ]:
# pima_descriptive_stats.py
# Descriptive statistics for Pima Indians Diabetes dataset (diabetes.csv)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# =========================
# Config
# =========================
CSV_PATH = "/kaggle/input/pima-indians-diabetes-database/diabetes.csv"
OUTPUT_DIR = "figs_pima"

# feature names (9 columns total)
ALL_COLS = [
    "Pregnancies",
    "Glucose",
    "BloodPressure",
    "SkinThickness",
    "Insulin",
    "BMI",
    "DiabetesPedigreeFunction",
    "Age",
    "Outcome",  # {0,1}
]

# zeros that should be treated as missing (physiologically implausible)
ZERO_AS_NA_COLS = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

# =========================
# Helpers
# =========================
def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def iqr_bounds(series: pd.Series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return q1, q3, iqr, lower, upper

def descriptive_table(df: pd.DataFrame, num_cols: list):
    """Return descriptive stats table with range, IQR, skew, kurtosis, and mode."""
    desc = df[num_cols].describe(percentiles=[0.25, 0.5, 0.75]).T
    desc["range"] = df[num_cols].max() - df[num_cols].min()
    desc["IQR"] = desc["75%"] - desc["25%"]
    desc["skew"] = df[num_cols].skew()
    desc["kurtosis"] = df[num_cols].kurt()
    # mode (cột có thể có nhiều mode) -> lấy mode[0] nếu có
    modes = {c: (df[c].mode().iat[0] if not df[c].mode().empty else np.nan) for c in num_cols}
    desc["mode"] = pd.Series(modes)
    return desc

def group_descriptive(df: pd.DataFrame, by_col: str, num_cols: list):
    """Descriptive by group (mean/median/std/Q1/Q3) with stable syntax across pandas versions."""
    def q1(s): return s.quantile(0.25)
    def q3(s): return s.quantile(0.75)
    q1.__name__ = "q1"; q3.__name__ = "q3"
    g = df.groupby(by_col)[num_cols].agg(['mean', 'median', 'std', q1, q3])
    # flatten columns: "feature_stat"
    g.columns = [f"{col}_{stat if isinstance(stat, str) else stat.__name__}" for col, stat in g.columns]
    return g

def outlier_report(df: pd.DataFrame, num_cols: list):
    rows = []
    for c in num_cols:
        q1, q3, iqr, lo, hi = iqr_bounds(df[c])
        mask = (df[c] < lo) | (df[c] > hi)
        rows.append((c, int(mask.sum()), float(lo), float(hi)))
    return pd.DataFrame(rows, columns=["feature", "n_outliers", "lower", "upper"]).sort_values("n_outliers", ascending=False)

# =========================
# 1) Load & basic checks
# =========================
df = pd.read_csv(CSV_PATH)
missing_cols = [c for c in ALL_COLS if c not in df.columns]
if missing_cols:
    raise ValueError(f"Missing expected columns: {missing_cols}\nFound: {list(df.columns)}")

# enforce numeric dtypes where applicable
for c in ALL_COLS:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# numeric features separate from target
NUM_COLS = [c for c in ALL_COLS if c != "Outcome"]

print("== RAW: Missing values per column ==")
print(df[ALL_COLS].isna().sum(), "\n")

# Class balance
print("== RAW: Outcome distribution ==")
print(df["Outcome"].value_counts().sort_index(), "\n")

# =========================
# 2) RAW descriptive stats
# =========================
raw_desc = descriptive_table(df, NUM_COLS)
print("== RAW: Descriptive statistics ==")
print(raw_desc.round(3), "\n")

# Selected percentiles
percentiles = [5, 25, 50, 75, 95]
raw_perc = pd.DataFrame(
    {c: np.percentile(df[c].dropna(), percentiles, method="linear") for c in NUM_COLS},
    index=[f"P{p}" for p in percentiles],
).T
print("== RAW: Selected percentiles ==")
print(raw_perc.round(3), "\n")

# RAW outliers (IQR)
raw_outliers = outlier_report(df, NUM_COLS)
print("== RAW: Outliers (IQR rule) ==")
print(raw_outliers, "\n")

# =========================
# 3) CLEAN: treat zeros as NA, impute median
# =========================
clean = df.copy()
for c in ZERO_AS_NA_COLS:
    clean.loc[clean[c] == 0, c] = np.nan  # zeros -> NA

print("== CLEAN: Missing after zero->NA ==")
print(clean[ALL_COLS].isna().sum(), "\n")

# Impute median for numeric cols (excluding Outcome)
for c in NUM_COLS:
    med = clean[c].median(skipna=True)
    clean[c] = clean[c].fillna(med)

# sanity: drop rows with Outcome NA (shouldn’t happen in this dataset)
clean = clean.dropna(subset=["Outcome"])

# =========================
# 4) CLEAN descriptive stats + by Outcome
# =========================
clean_desc = descriptive_table(clean, NUM_COLS)
print("== CLEAN: Descriptive statistics ==")
print(clean_desc.round(3), "\n")

clean_perc = pd.DataFrame(
    {c: np.percentile(clean[c].dropna(), percentiles, method="linear") for c in NUM_COLS},
    index=[f"P{p}" for p in percentiles],
).T
print("== CLEAN: Selected percentiles ==")
print(clean_perc.round(3), "\n")

clean_outliers = outlier_report(clean, NUM_COLS)
print("== CLEAN: Outliers (IQR rule) ==")
print(clean_outliers, "\n")

by_outcome = group_descriptive(clean, "Outcome", NUM_COLS).round(3)
print("== CLEAN: Descriptive by Outcome (mean/median/std/Q1/Q3) ==")
print(by_outcome, "\n")

# =========================
# 5) Visualizations
# =========================
ensure_dir(OUTPUT_DIR)

# 5.1 Histograms (CLEAN)
for c in NUM_COLS:
    plt.figure()
    clean[c].plot(kind="hist", bins=40, edgecolor="black")
    plt.title(f"{c} — Histogram (CLEAN)")
    plt.xlabel(c); plt.ylabel("count")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"hist_{c}.png"), dpi=120)
    plt.close()

# 5.2 Boxplot (CLEAN)
plt.figure(figsize=(10, 5))
clean[NUM_COLS].boxplot(rot=45)
plt.title("Boxplot — CLEAN numeric features")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "boxplot_clean.png"), dpi=120)
plt.close()

# 5.3 Correlation heatmap (CLEAN)
corr = clean[NUM_COLS + ["Outcome"]].corr(method="pearson")
plt.figure(figsize=(7, 6))
img = plt.imshow(corr.values, interpolation="nearest")
plt.title("Correlation heatmap (Pearson) — CLEAN")
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha="right")
plt.yticks(range(len(corr.index)), corr.index)
plt.colorbar(img, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "corr_heatmap_clean.png"), dpi=120)
plt.close()

# 5.4 Outcome balance bar
plt.figure()
clean["Outcome"].value_counts().sort_index().plot(kind="bar")
plt.title("Outcome distribution")
plt.xlabel("Outcome"); plt.ylabel("count")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "outcome_balance.png"), dpi=120)
plt.close()

print(f"Figures saved to ./{OUTPUT_DIR}/")


== RAW: Missing values per column ==
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64 

== RAW: Outcome distribution ==
Outcome
0    500
1    268
Name: count, dtype: int64 

== RAW: Descriptive statistics ==
                          count     mean      std     min     25%      50%  \
Pregnancies               768.0    3.845    3.370   0.000   1.000    3.000   
Glucose                   768.0  120.895   31.973   0.000  99.000  117.000   
BloodPressure             768.0   69.105   19.356   0.000  62.000   72.000   
SkinThickness             768.0   20.536   15.952   0.000   0.000   23.000   
Insulin                   768.0   79.799  115.244   0.000   0.000   30.500   
BMI                       768.0   31.993    7.884   0.000  27.300   32.000   
DiabetesPedi